#ETL com integração do Gemini
Este projeto é uma pipeline de **ETL** (Extração, Transformação e Exportação) que demonstra a integração e o processamento de dados usando diferentes APIs. O objetivo é coletar informações de pessoas, enriquecer esses dados com recomendações de filmes personalizadas e, por fim, exportar o resultado para um arquivo CSV.
O fluxo de trabalho inclui as seguintes etapas:

*   **Extração**: Os dados brutos são extraídos da **Faker API**, que fornece informações de perfil como nome, data de nascimento, cidade e país.
*   **Transformação**: Os dados extraídos são processados. Para cada pessoa, uma solicitação é enviada à **API do Gemini** para gerar três recomendações de filmes com base no seu perfil (nome, idade, cidade e país).
*   **Carga**: Os dados originais, agora enriquecidos com as recomendações de filmes, são exportados para um arquivo no formato CSV, prontos para serem usados em análises ou outras aplicações.

Este projeto serve como um exemplo prático de como orquestrar a coleta de dados de múltiplas fontes e usar a inteligência artificial para agregar valor a essas informações.

#Configurações iniciais

###Instalando a biblioteca que permitirá a utilização da **API do Gemini**

In [ ]:
!pip install -q -U google-generativeai

###Importando as bibliotecas que serão utilizadas no projeto

In [ ]:
import pandas as pd
import requests
import json
from google import genai
from google.colab import userdata

###Definindo url para utilização da **Faker API** e coletando a variável de ambiente da chave da **API do Gemini**

In [ ]:
API_URL = 'https://fakerapi.it/api/v2/'
API_KEY = userdata.get('GOOGLE_API_KEY')

#Extract - Extraindo dados da **Faker API**

###Definindo a função que fará a requisição na API

In [ ]:
def get_persons(quantity):
    url = f'{API_URL}persons?_quantity={quantity}'
    try:
      response = requests.get(url)
      return response.json().get('data') if response.status_code == 200 else None
    except requests.exceptions.RequestException as e:
      print(f'Ocorreu um erro de conexão: {e}')
      return None

###Adicionando as respostas da requisição em uma lista

In [ ]:
persons = []
QUANTITY = 10
persons = get_persons(QUANTITY)
for person in persons:
  print(person)

#Transform - Transformando os dados coletados

###Passando a chave da api para o objeto que fará a requisição

In [ ]:
client = genai.Client(api_key=API_KEY)

###Definindo a função que define o prompt e faz a requisição para o **Gemini**

In [ ]:
def get_message(person):
  prompt = f'''Recomende 3 filmes para {person["firstname"]} {person["lastname"]}
  que nasceu no dia {person["birthday"]} e mora em {person["address"]["country"]}
  na cidade {person["address"]["city"]}, limite a resposta a 200 caracteres sem negritos
  e escreva em inglês'''
  response = client.models.generate_content(
    model="gemini-2.5-flash", contents=prompt
  )
  return response.text

In [ ]:
for i, person in enumerate(persons):
  person['recomendations'] = get_message(person)
  print(i)

0
1
2
3
4
5
6
7
8
9


#Load - Exportando os dados enriquecidos

###Exportando os dados com a nova aba de recomendações para um csv

In [ ]:
df = pd.DataFrame(persons)
df.to_csv('persons_recomendations.csv')

In [ ]:
display(df)

,id,firstname,lastname,email,phone,birthday,gender,address,website,image,recomendations
0,1,Rickie,Abbott,eden.sipes@gmail.com,+19365623433,2023-09-13,other,"{'id': 1, 'street': '795 Stiedemann Heights', ...",http://lueilwitz.com,http://placeimg.com/640/480/people,"For Rickie, born 2023-09-13: My Neighbor Totor..."
1,2,Mabel,Hammes,walsh.marco@yahoo.com,+19493998212,1966-09-10,other,"{'id': 1, 'street': '7086 Medhurst Harbor Apt....",http://orn.com,http://placeimg.com/640/480/people,"For Mabel Hammes: Forrest Gump, The Shawshank ..."
2,3,Sigurd,Weber,ernie.rohan@gmail.com,+17162258310,2020-04-01,other,"{'id': 1, 'street': '6413 Minerva Corner Suite...",http://stehr.com,http://placeimg.com/640/480/people,"For Sigurd Weber, born 2020-04-01 in Camilamou..."
3,4,Charity,Hegmann,simeon.daniel@yahoo.com,+15642460496,1972-12-24,female,"{'id': 1, 'street': '50084 Braun Lake', 'stree...",http://hessel.com,http://placeimg.com/640/480/people,"For Charity Hegmann (1972, Iceland): The Shaws..."
4,5,Lonny,Langworth,maiya.hermann@powlowski.biz,+17862835502,1966-11-23,other,"{'id': 1, 'street': '92723 Gutmann Spring', 's...",http://blanda.com,http://placeimg.com/640/480/people,"For Lonny: The Shawshank Redemption, Forrest G..."
5,6,Dovie,D'Amore,laila09@kshlerin.com,+16418530199,1968-01-25,other,"{'id': 1, 'street': '9807 Mollie Views Suite 7...",http://bernier.info,http://placeimg.com/640/480/people,"For Dovie D'Amore (born 1968), I suggest: The ..."
6,7,Evie,Lind,emmy14@hotmail.com,+17604313023,1938-11-12,other,"{'id': 1, 'street': '49151 Reinger Forge Suite...",http://schuppe.com,http://placeimg.com/640/480/people,"Evie Lind, try Casablanca, The Sound of Music,..."
7,8,Shad,Hoppe,jessyca.stiedemann@hotmail.com,+19194528182,1937-05-12,other,"{'id': 1, 'street': '9295 Gene Radial Suite 62...",http://littel.com,http://placeimg.com/640/480/people,"For Shad Hoppe, try: Casablanca, The Godfather..."
8,9,Citlalli,Padberg,akautzer@heller.com,+15304826939,1999-07-27,female,"{'id': 1, 'street': '2391 Osinski Walk', 'stre...",http://lebsack.com,http://placeimg.com/640/480/people,"For Citlalli, I suggest Everything Everywhere ..."
9,10,Matilde,Kessler,emard.mona@grady.biz,+18584795869,1966-12-19,female,"{'id': 1, 'street': '6550 Marquis Course Apt. ...",http://paucek.com,http://placeimg.com/640/480/people,"For Matilde Kessler: Amelie, Paddington 2, Cas..."
